# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the IDs of the type and instance can be read from parameters of a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to a chatbot type (existing or new one)
- instance_id: Reference to chatbot instance (existing or new one)
- type_role: Role prompt of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context prompt of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Prompt that will be used to generate an initial message to the user (will be turned into a third prompt with role:system)

### 1. Preparation

##### 1.1. Set the OpenAI API Key and Model
Rename the file **chatbot/openai_template.py** to **chatbot/openai.py** and set the following keys.
- OPENAI_KEY = "your OpenAI API key in quotes"
- OPENAI_MODEL = "model name in quotes, e.g. gpt-3.5-turbo-16k"

##### 1.2. Install openai package

In [ ]:
%pip install openai

##### 1.3. Import Chatbot library

In [1]:
from chatbot.chatbot import Chatbot

### 2. Create a chatbot of type **health coach** for user **Daniel Müller**

#### 2.1. Prompt Engineering
To create a chatbot, we need to provide prompts. There are three prompts that can be used to customise the behaviour of the chatbot.
- **Role Prompt (type_role)**
    This prompt will be put at the beginning of the utterances sent to openai for completion. If you create multiple chatbots of the same type, this prompt will be the same for each one of them.
- **Context Prompt (instance_context)**
    This prompt will be put right after the role prompt. If you create multiple chatbots of the same type, this prompt will be used only for one of them.
- **Starter Prompt (instance_starter)**
    

Before we create our bot and store it in the dabase, we can edit the prompts and customise our bot or just use the once as prepared below.

##### type_role
Describes the general role of the chatbot.
    - What kind of persona should it enact/imitate. 
    - What is the context it acts in
    - What is the goal of the conversation.
    
This prompt will be turned into a first prompt with role:system

URL to be handed out to the user: If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

In [6]:
type_role = """
    As a digital therapy coach, check in daily with your patient to assess their well-being related to their chronic condition.
    Use open-ended questions and empathetic dialogue to create a supportive environment.
    Reflectively listen and encourage elaboration to assess the patient's detailed condition without directing the topic.
"""

##### instance_context 
Defines the context, in which the chatbot acts.
- Define how the answers of the chatbot should be formatted (here we added some html for the frontend)
- Define how the chatbot should be answering to the user


It will be turned into a second prompt with role:system.

In [ ]:
instance_context = """
    Meet Daniel Müller, 52, who is tackling obesity with a therapy plan that includes morning-to-noon intermittent fasting, thrice-weekly 30-minute swims, and a switch to whole grain bread.
"""

##### instance_starter

Will be used to generate an initial message to the user
- what should the chatbot start the conversation with
- ideally describe how the conversation shall continue after --> creates a more natural opening.

Will be turned into a final prompt with role:system

In [1]:

instance_starter = """
Jetzt, frage nach dem Namen und einem persönlichen Detail (z.B. Hobby, Beruf, Lebenserfahrung).
Verwende diese im geschlechtsneutralem Gespräch in Du-Form.
Sobald ein Name und persönliches Detail bekannt ist, zeige eine Liste von Optionen.
"""

#### Creating multiple instances of chatbot "Coach"
In the following, we can either create a bot for one user or for n users. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [ ]:
import uuid
import time

If you want to create more than one instance, change `number_of_instances`

In [ ]:
# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="4b9e7b25-08d7-4031-b485-4741ecd48dd1",
        user_id=user_id,
        type_name = "PuzzleBot",
        type_role = type_role,
        instance_context=instance_context,
        instance_starter=instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

- Enter your pythonanywhere username in the placeholder below.
- Change the type_id if you changed it above.

In [ ]:
pythonanywhere_username = "<ENTER YOUR PYTHONANYWHERE USERNAME HERE>"
type_id = "44b9e7b25-08d7-4031-b485-4741ecd48dd1"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))